In [180]:
# import xmltodict it didn't work for me
import re
import random
import pandas as pd

In [22]:
with open('../../rueter.github.io/emerald/myv-bidix/apertium-myv-rus.myv-rus-20221105-1945.dix', 'r') as f:
    bidix_lines = f.readlines()

In [27]:
bidix_lines[5:10]

['    </sdefs>\n',
 '    <section id="main" type="standard">\n',
 '        <e r="LR"><p><l>а ванькс<s n="adj"/></l><r>нечистоплотный<s n="adj"/></r></p></e>\n',
 '        <e r="LR"><p><l>а ванькс<s n="adj"/></l><r>нечистый<s n="adj"/></r></p></e>\n',
 '        <e r="LR"><p><l>а некшневикс<s n="adj"/></l><r>невиданный<s n="adj"/></r></p></e>\n']

In [29]:
bidix_lines[-10:]

['        <e r="LR"><p><l>ёртнемс<s n="vblex"/></l><r>бросать<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртнемс<s n="vblex"/></l><r>кидать<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртомс<s n="vblex"/></l><r>бросить<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртомс<s n="vblex"/></l><r>кинуть<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртомс<s n="vblex"/></l><r>швырнуть<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртомс<s n="vblex"/></l><r>забросить<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртотомс<s n="vblex"/></l><r>выбежать<s n="vblex"/></r></p></e>\n',
 '        <e r="LR"><p><l>ёртотомс<s n="vblex"/></l><r>выскочить<s n="vblex"/></r></p></e>\n',
 '    </section>\n',
 '</dictionary>\n']

Special processing:
* Comma or semicolon: alternative independent translations
* Slash: alternative translations within a phrase
* (): just remove

In [91]:
rus_letters = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя,; /-!?')

Here are all the special cases, printed

In [107]:
n_special = 0

raw_pairs = []

for line in bidix_lines[7:-2]:
    #dic = xmltodict.parse(line)
    lhs = re.findall('<l>(?P<lhs>.*)</l>', line)
    rhs = re.findall('<r>(?P<lhs>.*)</r>', line)
    assert len(lhs) == 1
    assert len(rhs) == 1
    lhs = re.sub('<s[^>]*>', '', lhs[0])
    rhs = re.sub('<s[^>]*>', '', rhs[0])
    
    for expr in [
        '\([\s,о]*\)', 
        '^\d\.',
        '; \d',
        '<>',
        '(перен|разг|тех|воен|спец|собир|бот|мат|безл|прям|полит)\.',
    ]:
        lhs = re.sub(expr, '', lhs).strip()
    
    lhs = re.sub('; \d', ';', lhs).strip()
        
    rhs = re.sub('\.', ',', rhs)
    
    # remove everything in brackets
    lhs = re.sub('\([^)]*\)', ' ', lhs).strip()
    rhs = re.sub('\([^)]*\)', ' ', rhs).strip()
    lhs = lhs.strip(' :.()…')
    
    if rhs == 'XX':
        continue
    
    if set(lhs.lower()).difference(rus_letters) or set(rhs.lower()).difference(rus_letters):
        print(lhs)
        print(rhs)
        print(line)
        print('-')
        n_special += 1
        continue
    
    raw_pairs.append((lhs, rhs))
        
print(n_special)

сэрей, покш;  покш питнеть. паро
высокий
        <e r="RL"><p><l>() сэрей, покш;  покш питнеть; 3.() паро<s n="adj"/></l><r>высокий<s n="adj"/></r></p></e>

-
велькска пандозь, доплатный:  доплатной сёрма
доплатной
        <e r="RL"><p><l>велькска пандозь, доплатный:  доплатной сёрма<s n="adj"/></l><r>доплатной<s n="adj"/></r></p></e>

-
эсь прядо/прянь,  монсень, эсень;  тонсеть, эсеть;  сонcензэ, эсензэ;  минсенек, эсенек;  тынсенк, эсенк;  сынсест, эсест
личный
        <e r="RL"><p><l>эсь прядо/прянь, (, ) монсень, эсень; () тонсеть, эсеть; () сонcензэ, эсензэ; () минсенек, эсенек; () тынсенк, эсенк; () сынсест, эсест<s n="adj"/></l><r>личный<s n="adj"/></r></p></e>

-
киськень:   киськень ормат
накожный
        <e r="RL"><p><l>киськень:   киськень ормат<s n="adj"/></l><r>накожный<s n="adj"/></r></p></e>

-
апак кекше, апак сёпо, весеменень неявиця/марявиця, неявиця\неявикс, виде паронь, виде
явный
        <e r="RL"><p><l>апак кекше, апак сёпо, весеменень неявиця/марявиця, неявиця\н

однокр. валомс
плеснуть
        <e r="RL"><p><l>однокр. валомс<s n="vblex"/></l><r>плеснуть<s n="vblex"/></r></p></e>

-
1  кепедемс, сэрейгавтомс, верев таргамс
повысить
        <e r="RL"><p><l>1 () кепедемс, сэрейгавтомс, верев таргамс<s n="vblex"/></l><r>повысить<s n="vblex"/></r></p></e>

-
однокр.  лазомс
полоснуть
        <e r="RL"><p><l>однокр. () лазомс<s n="vblex"/></l><r>полоснуть<s n="vblex"/></r></p></e>

-
уст. оймамс, куломс
почить
        <e r="RL"><p><l>уст. оймамс, куломс<s n="vblex"/></l><r>почить<s n="vblex"/></r></p></e>

-
савтомс:  ёжос савтомс
привести
        <e r="RL"><p><l>() савтомс:  ёжос савтомс<s n="vblex"/></l><r>привести<s n="vblex"/></r></p></e>

-
см. развернуть
разворачивать
        <e r="RL"><p><l>см. развернуть<s n="vblex"/></l><r>разворачивать<s n="vblex"/></r></p></e>

-
простякшномс, простямс, меремс «неемазонок»
распрощаться
        <e r="RL"><p><l>простякшномс, простямс, меремс «неемазонок»<s n="vblex"/></l><r>распрощаться<s n="vblex"/></r></p>

In [108]:
print(len(raw_pairs))

41748


In [153]:
def expand_alts(alts, pos=0):
    current_options = alts[pos]
    if pos+1 >= len(alts):
        return current_options
    future_options = expand_alts(alts, pos=pos+1)
    return [
        f"{w} {tail}"
        for w in current_options
        for tail in future_options
    ]

def expand(text):
    text = re.sub('[,;]', ',', text)
    text = re.sub('\s*/\s*', '/', text)
    result = []
    parts = text.split(',')
    for part in parts:
        part = part.strip()
        if not part:
            continue
        alts = []  # list of alternatives, for each word
        for w in part.split():
            if not w:
                continue
            alts.append(w.split('/'))
        result.extend(expand_alts(alts))
    return result
    
print(expand('серойшть-мериця / серойшть-мери раужо, маренго'))
print(expand('пацькамс, ваднемс, каргоцькавтомс, рудазсо ваднемс/пацькамс, рудазкавтомс'))
print(expand('лезэв, лезэнь максы;',))

['серойшть-мериця раужо', 'серойшть-мери раужо', 'маренго']
['пацькамс', 'ваднемс', 'каргоцькавтомс', 'рудазсо ваднемс', 'рудазсо пацькамс', 'рудазкавтомс']
['лезэв', 'лезэнь максы']


In [177]:
random.sample(raw_pairs, 5)

[('пелекизэ', 'пол-лета'),
 ('кичкерьгадомс, ве  сельме ёмавтомс', 'криветь'),
 ('превс путовкс/саевкс, урок', 'мораль'),
 ('таркаланго', 'постель'),
 ('валгомс', 'слезть')]

In [178]:
full_pairs = []
for lhs, rhs in raw_pairs:
    for left_text in expand(lhs):
        for right_text in expand(rhs):
            full_pairs.append((left_text, right_text))

In [179]:
print(len(full_pairs))

67944


In [181]:
df = pd.DataFrame(full_pairs)

In [182]:
df.columns = ['myv', 'rus']

In [195]:
df.sample(10)

,myv,rus
35893,усксема,развоз
63446,ве мельс прамс,сговориться
56563,саемс,одержать
29102,авай,матушка
2135,седе икеле невтезесь,вышеуказанный
38604,ангоркс,ссадина
14635,лацезь,сообразно
28520,мазыбаба,повитуха
51950,кокномс-мокномс,мямлить
9706,явовиця,различимый


In [198]:
df.to_csv('../../data-2023/parsed/apertium-myv-rus-20221105-1945.tsv', sep='\t', index=None)